**Installation**

In [1]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=f7ab18650f2bf916d0c73320f480881e1800c6ffd6524805e96dd8837fab79cf
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


**Libraries**

In [2]:
import nltk                             
import string
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
import pandas as pd
import os
import numpy as np

**Necessary Downloads**

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
INPUT_PATH = "/content/drive/MyDrive/Semester 3 IIITD/NLP/NLP_Project/Dataset/preprocessed_data"
#INPUT_PATH = "/content/drive/MyDrive/NLP/NLP_Project/Dataset/preprocessed_data"

In [19]:
train_data = pd.read_csv(os.path.join(INPUT_PATH,"divided_dataset/train.csv"))
train_data.head()

,Heading,Summary,Article,id
0,"un urges for maximum restraint, invokes simla ...","pakistan termed the indian action as ""unilater...","un chief invokes shimla agreement, calls for '...",1
1,"china, pak to finalise deal to develop sez und...","""the agreement will be finalised between khybe...","china, pak to finalise deal to develop sez und...",2
2,"covaxin effectively neutralises both alpha, de...",the top health research institute said that an...,"covaxin effectively neutralises both alpha, de...",3
3,man gets coronavirus twice with more severe sy...,a 25-year-old man in the us has caught coronav...,man gets coronavirus twice with more severe sy...,5
4,afghanistan president ghani flees to tajikista...,reports say that afghanistan president ashraf ...,ghani's close aides have also left the country...,6


In [20]:
test_data = pd.read_csv(os.path.join(INPUT_PATH,"divided_dataset/test.csv"))
test_data.head()

,Heading,Summary,Article,id
0,india opposes china's belt and road initiative...,the name of all member countries except india ...,"at sco, india refuses to back china's belt and...",0
1,"top white house officials buried cdc report, r...",the decision to shelve detailed advice from th...,"in this april 22, 2020, file photo president d...",4
2,us and china clash at un over south china sea ...,as india holds the council presidency this mon...,the united states and china clashed over beiji...,11
3,"us allows extra covid vaccine doses for some, ...",the food and drug administration ruled that tr...,vials for the moderna and pfizer covid-19 vacc...,13
4,pak minister claims threatening email was sent...,pakistan's information minister fawad chaudhry...,pakistan's information minister fawad chaudhry...,30


**Main code**

In [6]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'])

In [7]:
# return average of rouge 1 and rouge 2 f-measure for two sentences
def avg_rouge_score(sent1,sent2):
    r1 = scorer.score(sent1,sent2)['rouge1'].fmeasure
    r2 = scorer.score(sent1,sent2)['rouge2'].fmeasure
    return (r1 + r2)/2

In [8]:
# takes as input a list of sentences of doc
# and returns the sentence with highest rouge
def doc_sent_with_highest_rouge(doc):

    rouge_score = 0
    rouge_sent = ""

    original_doc = " ".join(doc)

    for sent in doc:
        rouge_score_sent = avg_rouge_score(sent,original_doc)
        if rouge_score_sent > rouge_score:
            rouge_score = rouge_score_sent
            rouge_sent = sent

    return rouge_sent

In [9]:
# find sentence of doc that has highest rouge with doc
def find_summary_sent(data):

    generated_summary = []

    heading = data['Heading']
    article = data['Article']

    for i in range(len(data)):
      heading_tokenize = nltk.sent_tokenize(heading[i])
      try:
        article_tokenize = nltk.sent_tokenize(article[i])
      except:
        article_tokenize = []
      final_tokenize = heading_tokenize + article_tokenize
      summary_sent = doc_sent_with_highest_rouge(final_tokenize)
      generated_summary.append(summary_sent)

    return generated_summary

**Train generated summary**

In [ ]:
# train_generated_summary = find_summary_sent(train_data)

In [ ]:
# train_generated_summary[:5]

**Test generated summary**

In [21]:
#test_generated_summary = find_summary_sent(test_data)

In [ ]:
# test_generated_summary[:5]

**Saving predictions**

In [22]:
# train_generated_df = pd.DataFrame(zip(train_generated_summary,train_data['id']),columns=['Summary','id'])
#test_generated_df = pd.DataFrame(zip(test_generated_summary,test_data['id']),columns=['Summary','id'])

In [14]:
RESULT_PATH = "/content/drive/MyDrive/Semester 3 IIITD/NLP/NLP_Project/Results/Smiti"
#RESULT_PATH = "/content/drive/MyDrive/NLP/NLP_Project/Results/Smiti"

In [15]:
# train_generated_df.to_csv(RESULT_PATH + "/DocSent_highest_rouge_train.csv",index=False)
#test_generated_df.to_csv(RESULT_PATH + "/DocSent_highest_rouge_test.csv",index=False)

In [16]:
train_generated_df = pd.read_csv(RESULT_PATH + "/DocSent_highest_rouge_train.csv")
test_generated_df = pd.read_csv(RESULT_PATH + "/DocSent_highest_rouge_test.csv")

**Evaluation of generated summaries**

In [17]:
# takes input a list of predicted and target summary
# and it returns final rouge score
def final_rouge_score(predicted,target):
    if len(predicted) != len(target):
        return "the dimensions of predicted and target must be same"
    rouge_score = 0
    for i in range(len(target)):
        rouge_score += avg_rouge_score(predicted[i],target[i])
    return rouge_score/len(target)

In [ ]:
final_rouge_score(train_generated_df['Summary'],train_data['Summary'])

0.2649250151534746

In [26]:
final_rouge_score(test_generated_df['Summary'],test_data['Summary'])

0.14426970124069058